In [9]:
from pyquil import Program, get_qc
import qulacs as q
from qulacs.gate import *
from qulacs import PauliOperator
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from qulacs.observable import create_split_observable, create_observable_from_openfermion_text
from qulacs import Observable, QuantumStateGpu, QuantumCircuit
import pyquil.api as api
import numpy as np
from numpy import linalg as LA
import math
import random
import itertools

In [10]:
hamiltonian_text = """
(3+0j) [I] + 
(-0.5+0j) [Z0] + 
(-0.5+0j) [Z6] + 
(0.5+0j) [Z0 Z6] + 
(-0.5+0j) [Z1] + 
(-0.5+0j) [Z7] + 
(0.5+0j) [Z1 Z7] + 
(-0.5+0j) [Z2] + 
(-0.5+0j) [Z8] + 
(0.5+0j) [Z2 Z8] + 
(-0.5+0j) [Z3] + 
(-0.5+0j) [Z9] + 
(0.5+0j) [Z3 Z9] + 
(-0.5+0j) [Z4] + 
(-0.5+0j) [Z10] + 
(0.5+0j) [Z4 Z10] + 
(-0.5+0j) [Z5] + 
(-0.5+0j) [Z11] + 
(0.5+0j) [Z5 Z11] + 
(-0.5+0j) [X0 X1] + 
(-0.5+0j) [Y0 Y1] + 
(-0.5+0j) [X0 Z1 X2] + 
(-0.5+0j) [Y0 Z1 Y2] + 
(-0.5+0j) [X0 Z1 Z2 Z3 Z4 X5] + 
(-0.5+0j) [Y0 Z1 Z2 Z3 Z4 Y5] + 
(-0.5+0j) [X1 X2] + 
(-0.5+0j) [Y1 Y2] + 
(-0.5+0j) [X1 Z2 Z3 X4] + 
(-0.5+0j) [Y1 Z2 Z3 Y4] + 
(-0.5+0j) [X2 X3] + 
(-0.5+0j) [Y2 Y3] + 
(-0.5+0j) [X3 X4] + 
(-0.5+0j) [Y3 Y4] + 
(-0.5+0j) [X3 Z4 X5] + 
(-0.5+0j) [Y3 Z4 Y5] + 
(-0.5+0j) [X4 X5] + 
(-0.5+0j) [Y4 Y5] + 
(-0.5+0j) [X6 X7] + 
(-0.5+0j) [Y6 Y7] + 
(-0.5+0j) [X6 Z7 X8] + 
(-0.5+0j) [Y6 Z7 Y8] + 
(-0.5+0j) [X6 Z7 Z8 Z9 Z10 X11] + 
(-0.5+0j) [Y6 Z7 Z8 Z9 Z10 Y11] + 
(-0.5+0j) [X7 X8] + 
(-0.5+0j) [Y7 Y8] + 
(-0.5+0j) [X7 Z8 Z9 X10] + 
(-0.5+0j) [Y7 Z8 Z9 Y10] + 
(-0.5+0j) [X8 X9] + 
(-0.5+0j) [Y8 Y9] + 
(-0.5+0j) [X9 X10] + 
(-0.5+0j) [Y9 Y10] + 
(-0.5+0j) [X9 Z10 X11] + 
(-0.5+0j) [Y9 Z10 Y11] + 
(-0.5+0j) [X10 X11] + 
(-0.5+0j) [Y10 Y11]
"""
hamiltonian = create_observable_from_openfermion_text(hamiltonian_text)
n_qubits = 12
print(hamiltonian.get_qubit_count())
print(hamiltonian.get_term_count())

12
55


In [11]:
model = [
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]}
]

model2 = [
    {"U": 2, "neighbors": [1], "hoppings": [0,1]},
    {"U": 2, "neighbors": [0], "hoppings": [1,0]}
]

model = [
    {"U": 2, "neighbors": [1,2,5], "hoppings": [0,1,1,0,0,1]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0]},
    {"U": 2, "neighbors": [0,1,3], "hoppings": [1,1,0,1,0,0]},
    {"U": 2, "neighbors": [2,4,5], "hoppings": [0,0,1,0,1,1]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1]},
    {"U": 2, "neighbors": [0,3,4], "hoppings": [1,0,0,1,1,0]}
]

model2 = [
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,1,0,1,0,0,0,1]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [1,0,1,0,0,0,1,0]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1,0,0]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0,0,0]},
    {"U": 2, "neighbors": [3,5,7], "hoppings": [0,0,0,1,0,1,0,1]},
    {"U": 2, "neighbors": [2,4,6], "hoppings": [0,0,1,0,1,0,1,0]},
    {"U": 2, "neighbors": [1,5,7], "hoppings": [0,1,0,0,0,1,0,1]},
    {"U": 2, "neighbors": [0,4,6], "hoppings": [1,0,0,0,1,0,1,0]}
]

t = t2 = t3 = 1
model3 = [
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,t,0,t,0,t2,0,0]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [t,0,t,0,0,0,t2,0]},
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,t,0,t,0,0,0,t2]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [t,0,t,0,t2,0,0,0]},
    {"U": 0, "neighbors": [3,5,7], "hoppings": [0,0,0,t2,0,t3,0,t3]},
    {"U": 0, "neighbors": [0,4,6], "hoppings": [t2,0,0,0,t3,0,t3,0]},
    {"U": 0, "neighbors": [1,5,7], "hoppings": [0,t2,0,0,0,t3,0,t3]},
    {"U": 0, "neighbors": [2,4,6], "hoppings": [0,0,t2,0,t3,0,t3,0]}
]

N = len(model)
print(N)

6


In [16]:
# I am confused on how the basis states for half filling of N sites translates to running the model. It seems like
# this part doesn't need to be edited, though, since the code is independent from pyquil.

N=6
def generate_basis_states(n):
    """
    Generates basis states for half filling of N sites
    """
    result = []
    int_result = []
    for bits in itertools.combinations(range(n * 2), n):
        # only choosing N bits from N * 2 bits, so half filling
        s = ['0'] * n * 2
        for bit in bits:
            # first n bits are up, second n are down
            # sites go 0 to N - 1 for each spin
            s[bit] = '1'
        
        if (''.join(s)[:n].count('1') == n / 2): # half filling
            int_result.append(int(''.join(s),2)) # integer representation of bit basis
            result.append(''.join(s))            # bit (string) basis
    # order ascending
    return (result[::-1], int_result[::-1])

(result, basis_states) = generate_basis_states(N)

In [17]:
def U_ansatz(n, param):
    """
    Creates a circuit for e^{i \theta_U H_U}
    """
    state = QuantumStateGpu(12)
    c = QuantumCircuit(12)
    for i in range(n):
        c.add_RZ_gate(i, param/4)
        c.add_RZ_gate(i+1, param/4)
        c.add_CNOT_gate(i, i+1)
        c.add_RZ_gate(i+n, -param/4)
        c.add_CNOT_gate(i, i+n)
    c.update_quantum_state(state)
    
    return c

In [18]:
def t_ansatz(s1, s2, param):
    """
    Creates a circuit for e^{i \theta_t H_t} for horizontal and vertical hoppings
    """
    
    state = QuantumStateGpu(12)
    c = QuantumCircuit(12)
    
    for i in range(s1+1, s2):
        if (i == s2-1):
            c.add_CZ_gate(i, i+1)
        else:
            c.add_CNOT_gate(i, i+1)
    
    c.add_H_gate(s1)
    c.add_H_gate(s2)
    c.add_CNOT_gate(s1, s2)
    c.add_RZ_gate(s2, param)
    c.add_CNOT_gate(s1, s2)
    c.add_H_gate(s1)
    c.add_H_gate(s2)
    c.add_RX_gate(s1, -np.pi/2)
    c.add_RX_gate(s2, -np.pi/2)
    c.add_CNOT_gate(s1, s2)
    c.add_RZ_gate(s2, param)
    c.add_CNOT_gate(s1, s2)
    c.add_RX_gate(s1, -np.pi/2)
    c.add_RX_gate(s2, -np.pi/2)
    
    for i in reversed(range(s1+1, s2)):
        if (i == s2-1):
            c.add_CZ_gate(i, i+1)
        else:
            c.add_CNOT_gate(i, i+1)
    
    return c

In [19]:
# This is from the pyquil code, I am not sure what this cell is doing so I don't know how to convert it to qulacs
# The main issue was from how the model was set up in pyquil. Since I have to explicitly write the hamiltonian,
# I still am figuring out how to work with the neighbors and hopping terms

simplified_h_t =  np.array([[ 0.0 for i in range(N)] for j in range(N)])
for i in range(N):
    for j in model[i]["neighbors"]:
        simplified_h_t[i][j] = -1.0
for i in range(N):
    simplified_h_t[i][N-1-i] = -1.51
print(simplified_h_t)
w2, v2 = LA.eigh(simplified_h_t)

for i in range(N):
    print(w2[i], v2[:,i])

[[ 0.   -1.   -1.    0.    0.   -1.51]
 [-1.    0.   -1.    0.   -1.51  0.  ]
 [-1.   -1.    0.   -1.51  0.    0.  ]
 [ 0.    0.   -1.51  0.   -1.   -1.  ]
 [ 0.   -1.51  0.   -1.    0.   -1.  ]
 [-1.51  0.    0.   -1.   -1.    0.  ]]
-3.509999999999998 [0.40824829 0.40824829 0.40824829 0.40824829 0.40824829 0.40824829]
-0.5100000000000011 [-0.0987518  -0.44325587  0.54200767  0.54200767 -0.44325587 -0.0987518 ]
-0.5100000000000006 [ 0.56884217 -0.36994265 -0.19889952 -0.19889952 -0.36994265  0.56884217]
-0.4900000000000016 [ 0.40824829  0.40824829  0.40824829 -0.40824829 -0.40824829 -0.40824829]
2.5099999999999993 [ 0.56236018 -0.39437581 -0.16798437  0.16798437  0.39437581 -0.56236018]
2.5100000000000016 [ 0.13070716  0.42166462 -0.55237178  0.55237178 -0.42166462 -0.13070716]


In [20]:
Q = v2[:,0:N//2].T

#Q = np.array([[0.5,0.5,0.5,0.5],[0.5,-0.5,-0.5,0.5]])
print(Q)
# convert to hopping direction matrix
for i in range(N):
    simplified_h_t[i][N-1-i] *= -1

from openfermion.utils import slater_determinant_preparation_circuit
from pyquil.quil import DefGate
from pyquil.quilatom import Parameter, quil_sin, quil_cos

def prepare_slater_determinant(Q): 
    """
    Prepares the Slater determinant as described in https://arxiv.org/pdf/1711.05395.pdf
    
    Args:
        Q: The (N_f x N) matrix Q with orthonormal rows which describes the Slater determinant to be prepared.
    Returns:
        p: A program that applies the sequence of Givens rotations returned 
            from slater_determinant_preparation_circuit
    """
    # Defining a controlled-RY gate
    theta = Parameter('theta')
    
    def CRY(control, target, angle):
        ry = qulacs.gate.RY(target, angle)
        cry = qulacs.gate.to_matrix_gate(ry)
        cry.add_control_qubit(control, target, angle)
    
    # Q is a (N_f x N) matrix
    N = Q[0].size
    N_f = len(Q)
    
    givens = slater_determinant_preparation_circuit(Q)

    def givens_rotation(tups, spin):
        """
        Performs Givens rotations
        
        Args:
            tups: tuple containing Givens rotations to be performed together
            spin: 0 represents up spin, and 1 represents down spin
        Returns:
            p: A program that applies the Givens rotations in tups
        """
        state = QuantumStateGpu(12)
        c = QuantumCircuit(12)

        for tup in tups:
            # where tup is (j, k, theta, phi)
            c.add_CNOT_gate(tup[1]+N*spin, tup[0]+N*spin)

            # controlled-RY
            c.CRY(tup[0]+N*spin, tup[1]+N*spin, tup[2])
            c.add_CNOT_gate(tup[1]+N*spin, tup[0]+N*spin)
            c.add_RZ_gate(tup[3], tup[1]+N*spin)
            c.update_quantum_state(state)
        
        return c
    
    state = QuantumStateGpu(12)
    c = QuantumCircuit(12)
    p += cry_definition
    
    # Fill first N_f orbitals for each spin
    for i in range(N_f):
        c.add_X_gate(i)
        c.add_X_gate(i+N)
    c.add_X_gate(3)
    c.add_X_gate(8)

  
    # Perform Givens rotations for up and down spins
    for rot in givens:
        p += givens_rotation(rot, 0)
        p += givens_rotation(rot, 1)
        
    return p

print(slater_determinant_preparation_circuit(Q))

[[ 0.40824829  0.40824829  0.40824829  0.40824829  0.40824829  0.40824829]
 [-0.0987518  -0.44325587  0.54200767  0.54200767 -0.44325587 -0.0987518 ]
 [ 0.56884217 -0.36994265 -0.19889952 -0.19889952 -0.36994265  0.56884217]]
[((2, 3, -0.7853981633974667, 0.0),), ((1, 2, -1.5707963267948966, 0.0), (3, 4, -1.5707963267948966, 0.0)), ((0, 1, -1.5707963267948966, 0.0), (2, 3, 0.7853981633974364, 0.0), (4, 5, -1.5707963267948966, 0.0)), ((1, 2, -1.5707963267948966, 0.0), (3, 4, -1.5707963267948966, 0.0)), ((2, 3, -0.785398163397442, 0.0),)]


In [ ]:
hops = []
for j in range(N):
    n = model[j]["neighbors"]
    for k in range(len(n)):
        # if a hopping is not already in the array i.e. don't add (3,0) if (0,3) is there
        if (j, n[k]) in hops or (n[k], j) in hops or j == n[k]:
            pass
        else:
            hops.append((j, n[k]))
h_hops = []
v_hops = []
for hop in hops:
    if (simplified_h_t[hop[0]][hop[1]] < 0):
        h_hops.append(hop)
    else:
        v_hops.append(hop)

def var_ansatz(params):
    """
    Prepares \sum_S [U_U(\theta_U / 2) U_h(\theta_h) U_v(\theta_v) U_U(\theta_U / 2)] | \Psi_I >
    Where | \Psi_I > is the Slater determinant for U = 0
    """
    S = 3
    p = Program()
    
    p += prepare_slater_determinant(Q)
    
    for i in range(S):
        # building U_u
        p += U_ansatz(N, params[3*i])

        # building U_t, horizontal and vertical
        # up spin horizontal hops
        for hop in h_hops:
            p += t_ansatz(hop[0], hop[1], params[3*i+1])
        # down spin horizontal hops
        for hop in h_hops:
            p += t_ansatz(hop[0]+N, hop[1]+N, params[3*i+1])

        # up spin vertical hops
        for hop in v_hops:
            p += t_ansatz(hop[0], hop[1], params[3*i+2])
        # down spin vertical hops
        for hop in v_hops:
            p += t_ansatz(hop[0]+N, hop[1]+N, params[3*i+2])  
    
        # building U_u
        p += U_ansatz(N, params[3*i])
        
    return p

In [ ]:
def cost(x):
    state = QuantumStateGpu(n_qubits)
    circuit = var_ansatz(x)
    circuit.update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

In [ ]:
from scipy.optimize import minimize

def expectation(x):
    """
    Expectation value for parameters, x
    """
    return cost(x)

def scipy_minimize(x):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """
    return minimize(expectation, x, method='powell')

def greedy_noisy_search(initial_state):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = cost(initial_state)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.1
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = cost(temp_params)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(initial_params=[]):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """
    points = [np.random.rand(9)*0.2 for i in range(6)]
    greedy_results = [greedy_noisy_search(x) for x in points]
    print("Done with greedy search")
    results = [scipy_minimize(i['x']) for i in greedy_results]
    print("Done with Powell's")
    res = min(results, key=lambda x:x.fun)
    print(res)
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x)
        temp_res = scipy_minimize(result['x'])
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.00001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [ ]:
res = global_variational()
print(res)